In [1]:
import pandas as pd
import numpy as np

from pymaid_creds import url, name, password, token
import pymaid
rm = pymaid.CatmaidInstance(url, token, name, password)

# pull predicted NT data
NT_data = pd.read_csv('data/NT/all-brain-presynapses_synapse_2023-09-08_preds.csv', index_col=0)

# pull all brain connectors
brain_skids = pymaid.get_skids_by_annotation('mw brain neurons') + pymaid.get_skids_by_annotation('mw brain accessory neurons')


INFO  : Global CATMAID instance set. Caching is ON. (pymaid)
INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)


In [2]:
# load connectors and determine fraction of each NT per neuron

connectors = pymaid.get_connector_details(NT_data.index.values)
connectors.index = connectors.connector_id

NT_data['skid'] = connectors.loc[np.intersect1d(NT_data.index.values, connectors.index)].presynaptic_to
NT_data['num'] = len(NT_data.index)*[1]
NT_neurons = NT_data.groupby(['skid', 'pred_label'], group_keys=True).count().unstack(fill_value=0).stack()

NT_fractions = NT_neurons/NT_neurons.groupby('skid').transform('sum')
NT_fractions = NT_fractions.unstack('pred_label')
NT_fractions.columns = [x[1] for x in NT_fractions.columns]

# add color based on majority vote
predictions = []
for i in NT_fractions.index:
    if(NT_fractions.loc[i, 'Acetylcholine']>0.5):
        prediction = 'predicted_Ach'
    if(NT_fractions.loc[i, 'GABA']>0.5):
        prediction = 'predicted_GABA'
    if(NT_fractions.loc[i, 'Glutamate']>0.5):
        prediction = 'predicted_Glutamate'
    if((NT_fractions.loc[i, 'Acetylcholine']<=0.5) & (NT_fractions.loc[i, 'GABA']<=0.5) & (NT_fractions.loc[i, 'Glutamate']<=0.5)):
        prediction = 'unclear'
    
    predictions.append(prediction)

NT_fractions['predictions'] = predictions

CN details:   0%|          | 0/170749 [00:00<?, ?it/s]

INFO  : Data for 170747 of 170749 unique connector IDs retrieved (pymaid)


In [3]:

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

fig = px.scatter_3d(NT_fractions, x='Acetylcholine', y='GABA', z='Glutamate', size=len(NT_fractions.index)*[1], size_max=8, color='predictions', color_discrete_sequence=['gray', 'red', 'purple', 'blue'] , width=1000, height=750)
fig.show()


In [4]:
NT_fractions.groupby('predictions').count().Acetylcholine

predictions
predicted_Ach          1780
predicted_GABA           90
predicted_Glutamate     716
unclear                 423
Name: Acetylcholine, dtype: int64

In [5]:
# assign NT to each skid based on the best guess with this current data

NT_data['skid'] = connectors.loc[np.intersect1d(NT_data.index.values, connectors.index)].presynaptic_to
NT_data['num'] = len(NT_data.index)*[1]
NT_neurons = NT_data.groupby(['skid', 'pred_label'], group_keys=True).count().unstack(fill_value=0).stack()

NT_fractions = NT_neurons/NT_neurons.groupby('skid').transform('sum')
NT_fractions = NT_fractions.unstack('pred_label')
NT_fractions.columns = [x[1] for x in NT_fractions.columns]

# add color based on majority vote
predictions = []
for i in NT_fractions.index:
    prediction = NT_fractions.loc[i, :].idxmax()    
    predictions.append(prediction)

NT_fractions['predictions'] = predictions

In [6]:
fig = px.scatter_3d(NT_fractions, x='Acetylcholine', y='GABA', z='Glutamate', size=len(NT_fractions.index)*[1], size_max=8, color='predictions', color_discrete_sequence=['purple', 'red', 'blue'] , width=1000, height=750)
fig.show()
NT_fractions.groupby('predictions').count().Acetylcholine

predictions
Acetylcholine    1974
GABA              162
Glutamate         873
Name: Acetylcholine, dtype: int64

In [8]:
NT_fractions.to_csv('data/NT/preliminary-NTs_2023-09-13.csv')

In [26]:
glutamate_predictions = NT_fractions[NT_fractions.predictions=='Glutamate'].index
acetylcholine_predictions = NT_fractions[NT_fractions.predictions=='Acetylcholine'].index
gaba_predictions = NT_fractions[NT_fractions.predictions=='GABA'].index

pymaid.add_annotations(list(gaba_predictions), 'mw predicted-gaba 2023-09-13')
pymaid.add_annotations(list(acetylcholine_predictions), 'mw predicted-ach 2023-09-13')
pymaid.add_annotations(list(glutamate_predictions), 'mw predicted-glut 2023-09-13')

pymaid.clear_cache()
pymaid.add_meta_annotations(['mw predicted-gaba 2023-09-13', 'mw predicted-ach 2023-09-13', 'mw predicted-glut 2023-09-13'], 'mw predicted NT')


INFO  : Cached cleared. (pymaid)


{'message': 'success',
 'annotations': [{'name': 'mw predicted NT',
   'id': 22453980,
   'entities': [22450968, 22453106, 22451131]}],
 'new_annotations': [22453980],
 'existing_annotations': []}